In [30]:
# from hmmlearn import hmm
import numpy as np
import pandas as pd
from collections import defaultdict
import datetime
import math
# import pykovy-master/ 
from pykovy.src.pykovy import chain
import matplotlib.pyplot as plt

In [19]:
df = pd.read_csv('data_sample/ap.csv')
df = df[['Client Username', 'Association Time', 'Map Location', 'Session Duration']]

In [20]:
# re-code userid
df['Client Username'] = pd.Categorical(df['Client Username'])
df['Client Username'] = df['Client Username'].cat.codes
# df = df.drop(['Client Username'], axis=1)

# recode map location
map_dict = {}
count = 0
for i in df['Map Location'].unique():
    map_dict[i] = count
    count += 1
df['Map Location'] = df['Map Location'].map(map_dict)


In [21]:
df

,Client Username,Association Time,Map Location,Session Duration
0,12,Sat Feb 24 00:38:41 PST 2018,0,7 hrs 25 min 57 sec
1,12,Sat Feb 24 08:04:39 PST 2018,0,8 hrs 10 min 25 sec
2,79,Sat Feb 24 13:16:49 PST 2018,1,2 hrs 58 min 15 sec
3,81,Fri Feb 23 16:38:44 PST 2018,1,1 hrs 48 min 10 sec
4,81,Fri Feb 23 18:26:55 PST 2018,1,10 min 26 sec
5,81,Sat Feb 24 13:42:56 PST 2018,1,2 hrs 32 min 9 sec
6,50,Sat Feb 24 11:00:26 PST 2018,2,15 min 36 sec
7,50,Sat Feb 24 11:16:03 PST 2018,2,31 min 8 sec
8,50,Sat Feb 24 11:47:11 PST 2018,2,10 min 23 sec
9,50,Sat Feb 24 11:57:34 PST 2018,2,1 hrs 13 min 30 sec


In [48]:
result = {}
result["client_username"] = [-1] * 3
result["client_username"][2] = 1
print(result)
print(24 % 24)

{'client_username': [-1, -1, 1]}
0


In [36]:
#from datetime import datetime
def convertMonth(month):
    if month == 'Jan': return 1
    elif month == 'Feb': return 2
    elif month == 'Mar': return 3
    elif month == 'Apr': return 4
    elif month == 'May': return 5
    elif month == 'Jun': return 6
    elif month == 'Jul': return 7
    elif month == 'Aug': return 8
    elif month == 'Sep': return 9
    elif month == 'Oct': return 10
    elif month == 'Nov': return 11
    elif month == 'Dec': return 12
    else: print('error input is not correct')
        
def convert_to_timeslot(duration):
    temp = duration.split()
    if len(temp) == 2: sess_duration = int(temp[0])
    elif len(temp) == 4: sess_duration = int(temp[0]) * 60 + int(temp[2])
    elif len(temp) == 6: sess_duration = (int(temp[0]) * 60 + int(temp[2])) * 60 + int(temp[4])
    else: print('error')
    return datetime.timedelta(seconds=sess_duration)

def convert_duration_to_seconds(duration):
    temp = duration.split()
    if len(temp) == 2: sess_duration = int(temp[0])
    elif len(temp) == 4: sess_duration = int(temp[0]) * 60 + int(temp[2])
    elif len(temp) == 6: sess_duration = (int(temp[0]) * 60 + int(temp[2])) * 60 + int(temp[4])
    else: print('error')
    return int(sess_duration)

def convert_to_datetime(time):
    temp = time.split()
    d_time = datetime.datetime(year=int(temp[5]), month=convertMonth(temp[1]), day=int(temp[2]), hour=int(temp[3].split(':')[0]), minute=int(temp[3].split(':')[1]), second=int(temp[3].split(':')[2]))
    return d_time

print(convert_to_datetime('Sat Feb 24 13:42:56 PST 2018'))
print(convert_duration_to_seconds('7 hrs 25 min 57 sec'))

2018-02-24 13:42:56
26757


In [29]:
#from datetime import datetime
# convert a time format
# Sat Feb 24 13:42:56 PST 2018
# to timestamp value
def convert_time_to_timestamp(input_time):
    datetime_format = convert_to_datetime(input_time)
    dt_obj = datetime.datetime.strptime(str(datetime_format),
                                        '%Y-%m-%d %H:%M:%S')
    return int(dt_obj.timestamp())
print(convert_time_to_timestamp('Sat Feb 24 13:42:56 PST 2018'))

1519508576


In [72]:
def convert_datetime_to_timestamp(input_time):
    return int(input_time.timestamp())
print(convert_datetime_to_timestamp(datetime.datetime(2018, 2, 24, 12, 2, 49)))

1519502569


In [73]:
def convert_timedelta_to_seconds(input_time):
    return int(input_time.total_seconds())
print(convert_timedelta_to_seconds(datetime.timedelta(seconds=1927)))

1927


In [32]:
def convert_timestamp_to_index(input_ts, start_ts, window_in_minutes):
    index = math.floor((input_ts - start_ts) / (window_in_minutes * 60))
    return index
print(convert_timestamp_to_index(1519508576, 1519462800, 60))

12


In [62]:
# data_list has same number of row as df.
# Each row [userid, map_loc, associate_time, associate_time+sess_dur, sess_dur].
def create_data_list(df):
    data_list = []
    c=0
    for i in range(df.shape[0]):
        #if c%500000 == 0:
        #    print(c)
        sess = convert_to_timeslot(df.iloc[i,3])
        s_time = convert_to_datetime(df.iloc[i,1])
        e_time = s_time + sess
        data_list.append([df.iloc[i,0], df.iloc[i,2], s_time, e_time, sess])
        c+=1
        
    data_list = sorted(data_list, key=lambda element: (element[0], element[2]))
    return data_list
data_list = create_data_list(df)
data_list

[[0,
  34,
  datetime.datetime(2018, 2, 24, 12, 2, 49),
  datetime.datetime(2018, 2, 24, 12, 34, 56),
  datetime.timedelta(seconds=1927)],
 [1,
  19,
  datetime.datetime(2018, 2, 23, 16, 33, 33),
  datetime.datetime(2018, 2, 23, 16, 38, 44),
  datetime.timedelta(seconds=311)],
 [1,
  7,
  datetime.datetime(2018, 2, 23, 16, 38, 44),
  datetime.datetime(2018, 2, 23, 16, 49, 46),
  datetime.timedelta(seconds=662)],
 [1,
  10,
  datetime.datetime(2018, 2, 23, 16, 49, 47),
  datetime.datetime(2018, 2, 23, 16, 55, 1),
  datetime.timedelta(seconds=314)],
 [1,
  19,
  datetime.datetime(2018, 2, 23, 16, 55, 14),
  datetime.datetime(2018, 2, 23, 23, 50, 20),
  datetime.timedelta(seconds=24906)],
 [1,
  19,
  datetime.datetime(2018, 2, 24, 0, 0, 51),
  datetime.datetime(2018, 2, 24, 11, 31, 31),
  datetime.timedelta(seconds=41440)],
 [1,
  19,
  datetime.datetime(2018, 2, 24, 11, 52, 23),
  datetime.datetime(2018, 2, 24, 13, 48, 8),
  datetime.timedelta(seconds=6945)],
 [1,
  9,
  datetime.dateti

In [64]:
# add up the session duration when the previous row has the same userid and location. 
# (this happen when you disconnect from the ap and reconnect again or your wifi session expire, etc.)
def preprocess_data_list(df, data_list):
    for i in reversed(range(1, df.shape[0])):
        if data_list[i][0] == data_list[i-1][0] and data_list[i][1] == data_list[i-1][1]: #and data_list[i][2] - data_list[i-1][3] <= timedelta(hours=5):
            data_list[i-1][4] += data_list[i][4]

    # loop each row, remove the next concecutive rows has the same userid and map_loc with the current row.
    # Because they are useless now
    final_data_list = []
    i = 0
    while i < df.shape[0]-1:
        if data_list[i][0] == data_list[i+1][0] and data_list[i][1] == data_list[i+1][1]:
            final_data_list.append(data_list[i])
            c = 2
            while i+c <= df.shape[0]-1 and data_list[i][0] == data_list[i+c][0] and data_list[i][1] == data_list[i+c][1]:
                c += 1
            i += c

        else:
            final_data_list.append(data_list[i])
            i += 1

    if not (final_data_list[-1][0] == data_list[-1][0] and final_data_list[-1][1] == data_list[-1][1]):
        final_data_list.append(data_list[-1])

    return final_data_list

final_data_list = preprocess_data_list(df, data_list)
final_data_list

[[0,
  34,
  datetime.datetime(2018, 2, 24, 12, 2, 49),
  datetime.datetime(2018, 2, 24, 12, 34, 56),
  datetime.timedelta(seconds=1927)],
 [1,
  19,
  datetime.datetime(2018, 2, 23, 16, 33, 33),
  datetime.datetime(2018, 2, 23, 16, 38, 44),
  datetime.timedelta(seconds=311)],
 [1,
  7,
  datetime.datetime(2018, 2, 23, 16, 38, 44),
  datetime.datetime(2018, 2, 23, 16, 49, 46),
  datetime.timedelta(seconds=662)],
 [1,
  10,
  datetime.datetime(2018, 2, 23, 16, 49, 47),
  datetime.datetime(2018, 2, 23, 16, 55, 1),
  datetime.timedelta(seconds=314)],
 [1,
  19,
  datetime.datetime(2018, 2, 23, 16, 55, 14),
  datetime.datetime(2018, 2, 23, 23, 50, 20),
  datetime.timedelta(days=1, seconds=42221)],
 [1,
  9,
  datetime.datetime(2018, 2, 24, 13, 48, 9),
  datetime.datetime(2018, 2, 24, 13, 53, 19),
  datetime.timedelta(seconds=310)],
 [2,
  41,
  datetime.datetime(2018, 2, 23, 17, 10, 58),
  datetime.datetime(2018, 2, 23, 17, 31, 54),
  datetime.timedelta(seconds=12547)],
 [3,
  62,
  dateti

In [66]:
# user_dict's key are each userid and value is a list of all rows of each user in final_data_list
def create_user_dict(final_data_list):
    user_dict = defaultdict(list)
    for row in final_data_list:
        user_dict[row[0]].append([row[2], row[1], row[4]])
        
#     for usr in user_dict:
#         user_dict[usr].sort()

    return user_dict
user_dict = create_user_dict(final_data_list)
user_dict

defaultdict(list,
            {0: [[datetime.datetime(2018, 2, 24, 12, 2, 49),
               34,
               datetime.timedelta(seconds=1927)]],
             1: [[datetime.datetime(2018, 2, 23, 16, 33, 33),
               19,
               datetime.timedelta(seconds=311)],
              [datetime.datetime(2018, 2, 23, 16, 38, 44),
               7,
               datetime.timedelta(seconds=662)],
              [datetime.datetime(2018, 2, 23, 16, 49, 47),
               10,
               datetime.timedelta(seconds=314)],
              [datetime.datetime(2018, 2, 23, 16, 55, 14),
               19,
               datetime.timedelta(days=1, seconds=42221)],
              [datetime.datetime(2018, 2, 24, 13, 48, 9),
               9,
               datetime.timedelta(seconds=310)]],
             2: [[datetime.datetime(2018, 2, 23, 17, 10, 58),
               41,
               datetime.timedelta(seconds=12547)]],
             3: [[datetime.datetime(2018, 2, 23, 16, 22, 44),
          

In [67]:
def remove_jitter(final_data_list):
    pass

In [71]:
def preprocess_and_create_user_dict(df):
    data_list = create_data_list(df)
    final_data_list = preprocess_data_list(df, data_list)
    #print(len(final_data_list), len(data_list))
#     final_data_list = remove_jitter(final_data_list)
    #print(len(final_data_list), len(data_list))
    user_dict = create_user_dict(final_data_list)
    return user_dict
user_dict = preprocess_and_create_user_dict(df)
user_dict

defaultdict(list,
            {0: [[datetime.datetime(2018, 2, 24, 12, 2, 49),
               34,
               datetime.timedelta(seconds=1927)]],
             1: [[datetime.datetime(2018, 2, 23, 16, 33, 33),
               19,
               datetime.timedelta(seconds=311)],
              [datetime.datetime(2018, 2, 23, 16, 38, 44),
               7,
               datetime.timedelta(seconds=662)],
              [datetime.datetime(2018, 2, 23, 16, 49, 47),
               10,
               datetime.timedelta(seconds=314)],
              [datetime.datetime(2018, 2, 23, 16, 55, 14),
               19,
               datetime.timedelta(seconds=73291)],
              [datetime.datetime(2018, 2, 24, 13, 48, 9),
               9,
               datetime.timedelta(seconds=310)]],
             2: [[datetime.datetime(2018, 2, 23, 17, 10, 58),
               41,
               datetime.timedelta(seconds=4716)]],
             3: [[datetime.datetime(2018, 2, 23, 16, 22, 44),
               62,


In [83]:
for usr in user_dict:
    print(usr)
    for i in range(0, len(user_dict[usr])):
        print(user_dict[usr][i][0])
        print(convert_datetime_to_timestamp(user_dict[usr][i][0]))
        print(user_dict[usr][i][1])
        print(convert_timedelta_to_seconds(user_dict[usr][i][2]))

0
2018-02-24 12:02:49
1519502569
34
1927
1
2018-02-23 16:33:33
1519432413
19
311
2018-02-23 16:38:44
1519432724
7
662
2018-02-23 16:49:47
1519433387
10
314
2018-02-23 16:55:14
1519433714
19
73291
2018-02-24 13:48:09
1519508889
9
310
2
2018-02-23 17:10:58
1519434658
41
4716
3
2018-02-23 16:22:44
1519431764
62
327
2018-02-23 16:28:11
1519432091
70
26209
2018-02-23 23:55:40
1519458940
71
3122
2018-02-24 01:20:38
1519464038
70
48920
2018-02-24 14:55:58
1519512958
62
317
2018-02-24 15:01:16
1519513276
72
311
2018-02-24 15:47:45
1519516065
73
1539
4
2018-02-24 14:45:12
1519512312
36
5289
5
2018-02-24 01:20:38
1519464038
15
13351
6
2018-02-23 16:38:44
1519432724
44
306
2018-02-23 16:49:47
1519433387
18
327
2018-02-23 16:55:14
1519433714
61
614
7
2018-02-24 01:04:57
1519463097
42
2500
8
2018-02-23 19:19:12
1519442352
61
619
9
2018-02-23 16:28:11
1519432091
77
45166
10
2018-02-23 16:33:33
1519432413
3
8969
11
2018-02-23 16:22:44
1519431764
24
2873
12
2018-02-24 00:38:41
1519461521
0
56182
13
20

3134
104
2018-02-23 17:38:53
1519436333
77
5676
105
2018-02-23 19:55:29
1519444529
17
6585
2018-02-23 22:42:22
1519454542
59
309
2018-02-24 14:13:56
1519510436
17
934
106
2018-02-23 17:50:11
1519437011
19
4701
107
2018-02-23 16:22:44
1519431764
57
960
108
2018-02-23 18:00:29
1519437629
74
2842
2018-02-23 20:37:00
1519447020
75
2177
2018-02-23 21:39:29
1519450769
62
620
2018-02-23 22:37:08
1519454228
9
314
2018-02-23 22:47:48
1519454868
26
2192
2018-02-24 03:06:02
1519470362
75
38494
109
2018-02-24 15:01:16
1519513276
56
622
110
2018-02-23 17:44:37
1519436677
24
18822
111
2018-02-24 11:05:38
1519499138
16
6597
112
2018-02-23 17:55:10
1519437310
22
8462
113
2018-02-23 18:47:52
1519440472
56
16932
114
2018-02-23 22:42:22
1519454542
17
35166
115
2018-02-23 18:10:51
1519438251
12
21349
2018-02-24 00:12:14
1519459934
26
324
2018-02-24 00:17:45
1519460265
12
310
2018-02-24 00:22:56
1519460576
26
15292
2018-02-24 04:37:48
1519475868
12
23582
2018-02-24 11:10:51
1519499451
9
309
116
2018-02-23 

In [87]:
# create a time slot 2d array each row is a user and each cols is a timeslot.
# for a particular user if there is no information on a particular time slot append(-1)
# this function is not finished
# Input:
# - df: dataframe
# - userdict: list of user id
# - start_timestamp: start time timestamp
# - end_timestamp: end time timestamp
# - window_in_minute: time window, use 60 (1 hour) first for simplicity
# 
# Output:
# - result: a hashmap (dictionary) where each map item is a 1D array where array 
#           indexes are indexed timeslot and values are "Map Location" (integer value)
#           -1 if there's no information for that timeslot
def create_time_slot_data(df, user_dict, start_timestamp, end_timestamp, window_in_minutes):
    # calculate number of index in each array
    # Formula:
    # number_of_time_entry = (diff of timestamp in second) / (number of seconds in that time window)
    number_of_time_entry = int((end_timestamp - start_timestamp) / (window_in_minutes * 60))
    
    # Addumption: start date at 0:0:0, end date at 23:59:59
    # 86400 = seconds in a day
    number_of_days = int((end_timestamp - start_timestamp) / 86400)
    
    number_of_time_entry_per_day = int(86400 / (window_in_minutes * 60))
    print(number_of_time_entry_per_day)
    result = {}
    #for i in range(df.shape[0]):
    for usr in user_dict:
        client_username = usr
        for i in range(0, len(user_dict[usr])):
        #print(user_dict[usr][i][0])
        #print(convert_datetime_to_timestamp(user_dict[usr][i][0]))
        #print(user_dict[usr][i][1])
        #print(convert_timedelta_to_seconds(user_dict[usr][i][2]))
        #client_username = df.iloc[i,0]
            association_time = convert_datetime_to_timestamp(user_dict[usr][i][0])
            map_location = user_dict[usr][i][1]
            session_duration = convert_timedelta_to_seconds(user_dict[usr][i][2])
        
            # 1/ If there's no entry for this user in result dict
            #    create a new 1d array with all -1 
            #    then change values base on association time and session duration
            # 2/ If user already in the result dict
            #    then change values base on association time and session duration
            # Caveats: there's maybe duplication, ignore it for now
            if client_username not in result:
                #time_entry_ts = start_timestamp
                #while time_entry_ts < end_timestamp:
                result[client_username] = [-1] * number_of_time_entry
                    #time_entry_ts += 86400
            # convert association time to timestamp
            # and calculate the total sesstion in timestamp
            association_time_ts = association_time
            end_association_time_ts = association_time_ts + session_duration
            if (association_time_ts > end_timestamp): 
                continue
            # TODO:
            # Need to handle the case where association timestamp < start_timestamp
            # in that case, some array entry might change
            start_index = convert_timestamp_to_index(association_time_ts, start_timestamp, window_in_minutes)
            end_index = convert_timestamp_to_index(end_association_time_ts, start_timestamp, window_in_minutes)
            # mark all the index from start index to end index to map location
            for i in range(start_index, end_index):
                result[client_username][i] = map_location
       
    #print(result)
    # Now we have a result dictionary without date
    # Do some more calculation to convert this to a dict with
    # {userid, date, list of timeslot}
    output = {}
    for user in result:
        output[user] = {}
        date_ts = start_timestamp
        output[user][date_ts] = [-1] * number_of_time_entry_per_day
        for i in range(0, number_of_time_entry):
            if (i != 0) and (i % number_of_time_entry_per_day == 0):
                date_ts += 86400
                output[user][date_ts] = [-1] * number_of_time_entry_per_day
            output[user][date_ts][i % number_of_time_entry_per_day] = result[user][i]
                
    return output

output = create_time_slot_data(df, user_dict, 1519376400, 1519545599, 60)
print(pd.DataFrame(output))

24
                                                          0    \
1519376400  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1519462800  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

                                                          1    \
1519376400  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1519462800  [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1...   

                                                          2    \
1519376400  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1519462800  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

                                                          3    \
1519376400  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1519462800  [70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 7...   

                                                          4    \
1519376400  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1519462800  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

                 

In [44]:
# helper function to lookup a user map location 
# on a given time
def lookup_map_location_by_time(data, client_username, input_time, start_time, window_in_minutes):
    time_ts = convert_time_to_timestamp(input_time)
    start_ts = convert_time_to_timestamp(start_time)
    index = convert_timestamp_to_index(time_ts, start_ts, window_in_minutes)
    return data[client_username][index]

print(lookup_map_location_by_time(output, 133, 'Sat Feb 24 01:20:38 PST 2018',
                                 'Sat Feb 23 01:00:00 PST 2018', 60))

75
